## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

application_df.drop(["EIN" , "NAME"] , axis =1 , inplace= True)


In [5]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [8]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T9' , 'T13' , 'T12' , 'T2' , 'T25' , 'T14' , 'T29' , 'T15' , 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [10]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [29]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#application_df['CLASSIFICATION'].value_counts()
cls_to_not_replace = ["C1000" , "C2000" , "C1200" , "C3000" , "C2100"]
application_df[~application_df['CLASSIFICATION'].isin(cls_to_not_replace)]['CLASSIFICATION']


13       C2700
22       C7000
44       C7200
48       C1700
69       C7000
         ...  
34165    C1700
34185    C7000
34212    C1700
34219    C7000
34226    C1800
Name: CLASSIFICATION, Length: 2261, dtype: object

In [30]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cls_to_not_replace = ["C1000" , "C2000" , "C1200" , "C3000" , "C2100"]

classifications_to_replace = application_df[~application_df['CLASSIFICATION'].isin(cls_to_not_replace)]['CLASSIFICATION']

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()



C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [39]:
# Convert categorical data to numeric with `pd.get_dummies`


cat_dummies = pd.get_dummies(application_df[['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION','SPECIAL_CONSIDERATIONS' , 'INCOME_AMT']])

numerical_columns = application_df[ 'ASK_AMT']

processed_df = pd.concat([cat_dummies , numerical_columns], axis=1)

processed_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,ASK_AMT
0,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,5000
1,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,108590
2,0,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,5000
3,0,0,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,6692
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,142590


In [40]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL']
X = processed_df

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [41]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [86]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=60, activation="relu", input_dim=42))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Third Hidden layer
nn.add(tf.keras.layers.Dense(units=40, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_49 (Dense)            (None, 60)                2580      
                                                                 
 dense_50 (Dense)            (None, 30)                1830      
                                                                 
 dense_51 (Dense)            (None, 40)                1240      
                                                                 
 dense_52 (Dense)            (None, 1)                 41        
                                                                 
Total params: 5,691
Trainable params: 5,691
Non-trainable params: 0
_________________________________________________________________


In [87]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [88]:
# Train the model

fit_model = nn.fit(X_train_scaled, y_train, epochs=200)


Epoch 1/200
804/804 [==============================] - 3s 2ms/step - loss: 0.5692 - accuracy: 0.7224
Epoch 2/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5538 - accuracy: 0.7314
Epoch 3/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5508 - accuracy: 0.7331
Epoch 4/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5491 - accuracy: 0.7340
Epoch 5/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5475 - accuracy: 0.7341
Epoch 6/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5474 - accuracy: 0.7357
Epoch 7/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5465 - accuracy: 0.7347
Epoch 8/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5455 - accuracy: 0.7365
Epoch 9/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5454 - accuracy: 0.7353
Epoch 10/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5441 - accura

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.5578 - acc: 0.7263
Loss: 0.557812534073699, Accuracy: 0.7262973785400391


In [ ]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE